In [2]:
import pandas as pd
import glob

In [41]:
username_relations = pd.read_csv('03_username_relations.csv')
username_relations.head()

,Unnamed: 0,Teamname,Tournament,Username
0,0,Atlanta Hustle,AUDL 2015,A Olsen
1,1,Atlanta Hustle,AUDL 2016,A Olsen
2,2,Atlanta Hustle,AUDL 2017,A Olsen
3,3,Atlanta Hustle,AUDL 2018,A Olsen
4,4,Atlanta Hustle,AUDL 2016,A Taylor


In [8]:
# roster2018 = pd.read_csv('02_roster2018.csv')
# roster2018.head()

,Unnamed: 0,team,PlayerName,Tournament,Teamname
0,0,radicals,Adam Drews,AUDL 2018,Madison Radicals
1,1,radicals,Andrew Meshnick,AUDL 2018,Madison Radicals
2,2,radicals,Seth Meyer,AUDL 2018,Madison Radicals
3,3,radicals,Pat Shriwise,AUDL 2018,Madison Radicals
4,4,radicals,Avery Johnson,AUDL 2018,Madison Radicals


In [34]:
# import difflib

# def MatchPlayerName(row):
#     roster = roster2018[roster2018.Teamname==row['Teamname']].PlayerName.unique()
#     matches = difflib.get_close_matches(row['Username'], roster)
#     return matches[0] if len(matches) > 0 else ''
    
# # username_relations['PlayerName'] = username_relations.apply(MatchPlayerName)
# # username_relations['PlayerName'] = username_relations.apply(MatchPlayerName,axis=1)

In [53]:
matched_2018 = pd.read_excel('2018.07.11_2018RostersUAMatchup_Final.xlsx').sort_values(['Teamname','Username']).reset_index(drop=True)
matched_2018.head()

,Teamname,Username,PlayerName
0,Atlanta Hustle,A Olsen,Anders Olsen
1,Atlanta Hustle,A Taylor,Austin Taylor
2,Atlanta Hustle,Allan L,Allan Laviolette
3,Atlanta Hustle,Batson,Sam Batson
4,Atlanta Hustle,Big Mike,Mike Tillotson


In [78]:
matched_2017 = pd.read_csv('audlrosters2017.csv',
                           header=None).rename(columns={1:'Teamname',
                                                        5:'PlayerName',
                                                        6:'Username'})[['Teamname',
                                                                        'PlayerName',
                                                                        'Username']].drop_duplicates().replace('San Francisco Flamethrowers','San Francisco FlameThrowers')
matched_2017.head()

,Teamname,PlayerName,Username
0,Dallas Roughnecks,Kurt Gibson,Kurt
1,Atlanta Hustle,Ryan Archibald,Archie
2,Atlanta Hustle,Evan Boecking,Boecking
3,Atlanta Hustle,JP Burns,JP Burns
4,Atlanta Hustle,Josh Bush,Bush


In [79]:
[i for i in matched_2017.Teamname.unique() if i not in matched_2018.Teamname.unique()]

['Jacksonville Cannons', 'Vancouver Riptide']

In [80]:
[i for i in matched_2018.Teamname.unique() if i not in matched_2017.Teamname.unique()]

['Tampa Bay Cannons']

In [84]:
matched = pd.merge(matched_2017,
                    matched_2018,
                    right_on=['Teamname','Username'],
                    left_on=['Teamname','Username'],
                    how='outer').replace('Nashville Nightwatch','Nashville NightWatch')
matched['PlayerName'] = matched.PlayerName_x.fillna(matched.PlayerName_y) # Use 2017 if conflicting
matched = matched[['Teamname','PlayerName','Username']]
matched.head()

,Teamname,PlayerName,Username
0,Dallas Roughnecks,Kurt Gibson,Kurt
1,Atlanta Hustle,Ryan Archibald,Archie
2,Atlanta Hustle,Evan Boecking,Boecking
3,Atlanta Hustle,JP Burns,JP Burns
4,Atlanta Hustle,Josh Bush,Bush


In [86]:
[i for i in matched.Teamname.unique() if i not in username_relations.Teamname.unique()]

[]

In [85]:
[i for i in username_relations.Teamname.unique() if i not in matched.Teamname.unique()]

['Charlotte Express',
 'Cincinnati Revolution',
 'Rochester Dragons',
 'Salt Lake Lions',
 'Seattle Raptors']

In [87]:
username_playername = pd.merge(username_relations,
                                 matched,
                                left_on=['Teamname','Username'],
                                right_on=['Teamname','Username'],
                                how='left')

# username_playername[username_playername.Tournament=='AUDL 2018']
# username_playername.dropna()
# username_playername[pd.isnull(username_playername.PlayerName)&(username_playername.Teamname=='Atlanta Hustle')]

username_playername.to_csv('04_username_playername_relation.csv')

In [88]:
len(username_playername.dropna()) / len(username_playername)

0.46609951018303686